In [1]:
!git clone --depth 1 https://github.com/tensorflow/models
!pip install protobuf
!pip install absl-py
!pip install tensorflow==2.8.0
!pip install pyqtwebengine==5.13
!pip install wget
import re
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

fatal: destination path 'models' already exists and is not an empty directory.


In [2]:
# Run Model Bulider Test file, just to verify everything's working properly
!python models/research/object_detection/builders/model_builder_tf2_test.py

In [3]:
%%bash
cat <<EOF >> labelmap.txt
biker
car
pedestrian
trafficLight
trafficLight-Green
trafficLight-GreenLeft
trafficLight-Red
trafficLight-RedLeft
trafficLight-Yellow
trafficLight-YellowLeft
truck
EOF

In [4]:
# Download data conversion scripts
!python -m wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/cre

Traceback (most recent call last):
  File "/Users/teddys2k/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/teddys2k/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/teddys2k/anaconda3/lib/python3.8/site-packages/wget.py", line 568, in <module>
    filename = download(args[0], out=options.output)
  File "/Users/teddys2k/anaconda3/lib/python3.8/site-packages/wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
  File "/Users/teddys2k/anaconda3/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/Users/teddys2k/anaconda3/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/teddys2k/anaconda3/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File

In [5]:
!python3 create_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


In [6]:
train_record_fname = '/train.tfrecord'
val_record_fname = '/val.tfrecord'
label_map_pbtxt_fname = '/labelmap.pbtxt'

In [7]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [8]:
%cd /Users/teddys2k/Desktop/Tensorflow-model
%ls

/Users/teddys2k/Desktop/Tensorflow-model
Setup.ipynb     create_csv.py   labelmap.txt
Untitled.ipynb  images/         models/


In [9]:
# Download pre-trained model weights
%cd /Users/teddys2k/Desktop/Tensorflow-model/models/mymodel
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
if chosen_model == 'centernet-mobilenet-v2':
  download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20210210/' + pretrained_checkpoint

!python -m wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
if chosen_model == 'centernet-mobilenet-v2':
  !cp models/mymodel/centernet_mobilenetv2_fpn_od/pipeline.config /Users/teddys2k/Desktop/Tensorflow-model/models/mymodel

else:
  !python -m wget {download_config}
#!wget {download_config}

/Users/teddys2k/Desktop/Tensorflow-model/models/mymodel
100% [....................................................] 46042990 / 46042990
Saved under ssd_mobilenet_v2_320x320_coco17_tpu-8.tar (15).gz
100% [............................................................] 4484 / 4484
Saved under ssd_mobilenet_v2_320x320_coco17_tpu-8 (15).config


In [10]:
# Set training parameters for the model
num_steps = 40000
batch_size = 8

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/Users/teddys2k/Desktop/Tensorflow-model/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/Users/teddys2k/Desktop/Tensorflow-model/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)

In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /Users/teddys2k/Desktop/Tensorflow-model/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)
      
      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)
    
    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)